In [46]:
import torch
import torch.nn as nn
import torch.optim as optim


In [47]:
class PoissonRegression(nn.Module):
    def __init__(self, input_dim):
        super(PoissonRegression, self).__init__()
        self.linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        return torch.exp(self.linear(x))


generate data

In [48]:
import numpy as np
import random

# Generate the artificial data

n_total = int(20000)
K = 100
X = np.column_stack((np.ones(n_total), np.random.rand(n_total * (K - 1)).reshape(n_total, K - 1)))
b0 = 1.0 * np.ones(K) / K
y = np.random.poisson(np.exp(X.dot(b0)))


In [49]:
# Split the data into train and test sets
n = len(y)
test_indices = random.sample(range(n), round(0.2 * n))

y_test = y[test_indices]
X_test = X[test_indices, :]

y_train = np.delete(y, test_indices)
X_train = np.delete(X, test_indices, axis=0)

# convert X_train into a pytorch tensor
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()

In [50]:
model = PoissonRegression(K)
optimizer = optim.SGD(model.parameters(), lr=1)


In [51]:
def poisson_nll_loss(y_pred, y_true):
    return torch.mean(y_pred - y_true * torch.log(y_pred)) # this is the correct formula. 
# y_pred is the linear index, y_true is the true value.




In [52]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

for epoch in range(100):  # number of epochs
    optimizer.zero_grad()
    y_pred = model(X_train) # the liner index
    loss = poisson_nll_loss(y_pred, y_train)
    loss.backward()
    optimizer.step()

    if epoch % 1 == 0:  # print every 100 epochs
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')


Epoch 1, Loss: 1.0363727807998657
Epoch 2, Loss: 0.9319944381713867
Epoch 3, Loss: 0.8806751370429993
Epoch 4, Loss: 0.8586091995239258
Epoch 5, Loss: 0.8500959873199463
Epoch 6, Loss: 0.8470416069030762
Epoch 7, Loss: 0.8459743857383728
Epoch 8, Loss: 0.8455837965011597
Epoch 9, Loss: 0.8454144597053528
Epoch 10, Loss: 0.8453155755996704
Epoch 11, Loss: 0.8452390432357788
Epoch 12, Loss: 0.8451696634292603
Epoch 13, Loss: 0.8451026678085327
Epoch 14, Loss: 0.8450363874435425
Epoch 15, Loss: 0.8449705243110657
Epoch 16, Loss: 0.8449048399925232
Epoch 17, Loss: 0.8448394536972046
Epoch 18, Loss: 0.8447741866111755
Epoch 19, Loss: 0.8447092771530151
Epoch 20, Loss: 0.8446443676948547
Epoch 21, Loss: 0.8445796966552734
Epoch 22, Loss: 0.8445151448249817
Epoch 23, Loss: 0.844450831413269
Epoch 24, Loss: 0.8443866968154907
Epoch 25, Loss: 0.844322681427002
Epoch 26, Loss: 0.8442590236663818
Epoch 27, Loss: 0.8441953063011169
Epoch 28, Loss: 0.8441319465637207
Epoch 29, Loss: 0.8440686464309

In [53]:
# print the coefficients of the model

# print(model.state_dict()['linear.weight'])
print(model.state_dict())

OrderedDict([('linear.weight', tensor([[ 0.1140,  0.0155, -0.0001,  0.0363,  0.0075,  0.0911,  0.0637, -0.0624,
          0.0763,  0.0091, -0.0147,  0.0587,  0.0611, -0.0640,  0.0667, -0.0058,
         -0.0699,  0.0514,  0.0667,  0.0669,  0.0357, -0.0159, -0.0748, -0.0261,
         -0.0469, -0.0725,  0.0733, -0.0590,  0.0912,  0.0842,  0.0227, -0.0502,
          0.0171, -0.0493, -0.0068,  0.0018, -0.0082,  0.0601, -0.0337, -0.0524,
          0.0178,  0.0744, -0.0777, -0.0044, -0.0092,  0.0883,  0.0664,  0.0108,
          0.0016, -0.0282, -0.0497,  0.0355,  0.0188, -0.0323,  0.0621,  0.0882,
          0.0454, -0.0203,  0.0104,  0.0652, -0.0275, -0.0463,  0.0512, -0.0557,
          0.0315,  0.0067,  0.0731, -0.0751,  0.0109,  0.0971, -0.0200,  0.0061,
         -0.0675, -0.0673,  0.0191,  0.0688, -0.0607, -0.0754, -0.0542, -0.0682,
         -0.0561,  0.0296,  0.0826,  0.0350, -0.0670,  0.0466, -0.0114, -0.0435,
          0.0051,  0.0681,  0.0737, -0.0708, -0.0377,  0.0278, -0.0063, -0.036